In [1]:
import fastf1 as ff1
from fastf1 import plotting
from fastf1.core import Laps

import pandas as pd
import numpy as np

In [40]:
# Enable the cache by providing the name of the cache folder, speed up
ff1.Cache.enable_cache('cache')

# Setup plotting, setup the plot (bg: black, ...)
plotting.setup_mpl()

In [41]:
ff1.Cache.offline_mode(True)

In [92]:
FILE_CIRCUITS = 'f1_unique_circuits_complete.csv'

circuit_info = pd.read_csv(FILE_CIRCUITS, delimiter=",")

In [4]:
session = ff1.get_session(2025, 'Monza', 'Q')
session.load()

core           INFO 	Loading data for Italian Grand Prix - Qualifying [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '81', '16', '44', '63', '12', '5', '14', '22', '87', '27', '55', '23', '31', '6', '18', '43', '10', '30']


In [ ]:
driver = 'VER'
# fastest lap of the driver
fastest_lap = session.laps.pick_drivers(driver).pick_fastest()
telemetry_driver = fastest_lap.get_telemetry().add_distance()


In [6]:
max(telemetry_driver['Speed'])

348.0

In [7]:
def calculate_physics(physics_df):
    # Convert speed from km/h to m/s and add it as a new column
    physics_df['v_m/s'] = physics_df['Speed'] / 3.6

    # Convert Time to seconds (float) and add it as a new column
    physics_df['time_s'] = physics_df['Time'] / np.timedelta64(1, 's')

    # Calculate longitudinal (forward/backward) acceleration (ax)
    # This is the derivative of velocity (v) with respect to time (t), or a = dv/dt
    # np.gradient() is a standard way to compute this from arrays of data.
    physics_df['ax_m/s^2'] = np.gradient(physics_df['v_m/s']) / np.gradient(physics_df['time_s'])

    # Smooth the acceleration data.
    # Telemetry data is often "noisy" (spiky), which makes derivatives very unstable.
    # np.convolve() applies a simple 3-point moving average to smooth out the spikes,
    # giving a more realistic acceleration value.
    physics_df['ax_smooth_m/s^2'] = np.convolve(
        physics_df['ax_m/s^2'], 
        np.ones((3,))/3, 
        mode='same'
    )

    # --- MODEL I: STATIC WEIGHT ONLY ---
    m = 798    # kg (Mass)
    g = 9.81   # m/s^2 (Gravity)
    staticWeightTot = m * g
    fractionWeightFront = 0.46

    # Calculate the static weight (in Newtons) on the front and rear axles
    staticWeightFront = staticWeightTot * fractionWeightFront
    staticWeightRear = staticWeightTot - staticWeightFront

    # Add the static loads to the DataFrame
    physics_df['loadFront_static_N'] = staticWeightFront
    physics_df['loadRear_static_N'] = staticWeightRear

    # --- MODEL II: INERTIAL LOAD TRANSFER ADDED ---
    CoGheight = 0.25    # m (Center of Gravity height)
    wheelbase = 3.6     # m (Wheelbase)

    # Calculate the *change* in load (deltaLoad) caused by longitudinal acceleration (ax)
    # This is the core "load transfer" equation.
    # If ax_smooth is positive (accelerating), deltaLoad becomes negative.
    physics_df['deltaLoad_Inertial_N'] = -CoGheight / wheelbase * m * physics_df['ax_smooth_m/s^2']

    # Add the inertial load to the static load
    # Calculate the new, dynamic load on the front and rear axles
    # Front Axle:
    #   - If accelerating (deltaLoad < 0), the front load *decreases*.
    physics_df['loadFront_Inertial_N'] = physics_df['loadFront_static_N'] + physics_df['deltaLoad_Inertial_N']
    # Rear Axle:
    #   - If accelerating (deltaLoad < 0), subtracting a negative *increases* the rear load.
    physics_df['loadRear_Inertial_N'] = physics_df['loadRear_static_N'] - physics_df['deltaLoad_Inertial_N']

    # --- MODEL III: AERODYNAMIC EFFECTS ADDED ---
    rho = 1.225         # kg/m^3 (Standard air density at sea level)
    CdA = 1.7           # Drag Coefficient * Frontal Area. Measures total air resistance.
    efficiency = 3.7      # Aerodynamic efficiency (Lift/Drag ratio). For F1, this is Downforce/Drag.
    ClA = efficiency*CdA  # Downforce Coefficient * Frontal Area. Measures total downforce.
    dragHeight = 0.5    # m (Height of drag force application)

    # Calculate Drag and Downforce (which depend on speed)
    # Calculate Drag Force (in Newtons)
    # Formula: Fd = 0.5 * rho * v^2 * CdA
    # Force increases with the *square* of velocity (v).
    physics_df['dragForce_N'] = 0.5 * CdA * rho * np.square(physics_df['v_m/s'])
    # Calculate Downforce (in Newtons)
    # Formula: Fl = 0.5 * rho * v^2 * ClA
    # This force also increases with the *square* of velocity.
    physics_df['downForce_N'] = 0.5 * ClA * rho * np.square(physics_df['v_m/s'])

    # Distribute the aerodynamic forces
    fractionDownforceFront = 0.4
    physics_df['downForceFront_N'] = physics_df['downForce_N'] * fractionDownforceFront
    physics_df['downForceRear_N'] = physics_df['downForce_N'] * (1 - fractionDownforceFront)

    # Calculate the load transfer caused *by drag*.
    # Drag pulls the car backward from a point *above* the ground (dragHeight),
    # which creates a moment that also shifts load to the rear axle.
    physics_df['deltaLoad_Drag_N'] = -dragHeight / wheelbase * physics_df['dragForce_N']

    # FINAL FRONT LOAD:
    # Model II load + downforce on the front + load transfer from drag
    physics_df['loadFront_AeroModel_N'] = (
        physics_df['loadFront_Inertial_N'] + 
        physics_df['downForceFront_N'] + 
        physics_df['deltaLoad_Drag_N']
    )
    # FINAL REAR LOAD:
    # Model II load + downforce on the rear - load transfer from drag
    physics_df['loadRear_AeroModel_N'] = (
        physics_df['loadRear_Inertial_N'] + 
        physics_df['downForceRear_N'] - 
        physics_df['deltaLoad_Drag_N']
    )

    return physics_df

# Longitudinal and Latitudinal acceleration

In [8]:
def convert_to_g(accelerations_ms2):
    g = 9.81
    return accelerations_ms2 / g


In [66]:
import numpy as np
import pandas as pd # Needed for type checking/conversion

def smooth_derivative(t_in, v_in, method="smooth"):
    """
    Computes a smooth estimation of a derivative (dv/dt) using a multi-point
    finite difference stencil to reduce noise amplification.
    """
    t = t_in.copy()
    v = v_in.copy()
    
    # (0) Prepare inputs: Ensure Time is transformed to seconds (a float array)
    try:
        # Check if t is a Pandas Series of timedelta objects
        if isinstance(t, pd.Series) and t.dtype == 'timedelta64[ns]':
            t = np.array([td.total_seconds() for td in t])
        else:
             # Assume it's a list/array of timedeltas needing conversion
             t = np.array([td.total_seconds() for td in t])
    except:
        # If it throws an error, it's likely already a float/int array (e.g., Distance)
        t = np.array(t)
        
    v = np.array(v)
    
    # Assert they have the same size and initialize output array
    assert t.size == v.size
    n = t.size
    dvdt = np.zeros(n)
    
    if n < 6:
        # Fallback to simple gradient if not enough points for the stencil
        return np.gradient(v, t)

    # (1) Manually compute points out of the stencil (boundary conditions)
    # The first few and last few points can't use the full stencil, so we use
    # a simpler, but stable, forward/backward difference approximation.
    # Helper function to safely calculate the slope (dv/dt)
    def safe_slope(v_num, t_den):
        # Check if the time difference is effectively zero
        if t_den == 0:
            return 0.0  # Return 0 derivative if time difference is zero
        return v_num / t_den

    # Boundary conditions: Apply the safe_slope helper to all boundary calculations
    dvdt[0] = safe_slope(v[1] - v[0], t[1] - t[0])
    dvdt[1] = safe_slope(v[2] - v[0], t[2] - t[0])
    dvdt[2] = safe_slope(v[3] - v[1], t[3] - t[1])
    
    dvdt[n-1] = safe_slope(v[n-1] - v[n-2], t[n-1] - t[n-2])
    dvdt[n-2] = safe_slope(v[n-1] - v[n-3], t[n-1] - t[n-3])
    dvdt[n-3] = safe_slope(v[n-2] - v[n-4], t[n-2] - t[n-4])

    # (2) Compute the rest of the points using the stencil
    if (method=='smooth'):
        # This uses a weighted average of multiple centered differences
        # (up to 3 points away) to achieve maximum noise suppression.
        c = [5./32., 4./32., 1./32.]
        for i in range(3, n-3):
            for j in range(1, 4):
                v_num = v[i+j] - v[i-j]
                t_den = t[i+j] - t[i-j]
                # Only add to the derivative if the time difference is non-zero
                if t_den != 0:
                    dvdt[i] += 2 * j * c[j-1] * (v_num / t_den)

    elif (method == 'centered'):
        # Fallback for a standard (but still 2-point) centered difference
        for i in range(3, n-3):
            v_num = v[i+1] - v[i-1]
            t_den = t[i+1] - t[i-1]
            
            # Only assign the derivative if the time difference is non-zero
            if t_den != 0:
                dvdt[i] = v_num / t_den
            else:
                dvdt[i] = 0.0 # Assign zero derivative if time difference is zero
            
    return dvdt

In [10]:
import math

def truncated_remainder(dividend, divisor):
    """
    Computes a truncated remainder, ensuring consistent mathematical behavior
    especially for negative numbers, unlike standard Python's modulo operator.
    """
    
    # Calculate the integer part of the division (the quotient)
    # Using specific logic to ensure 'truncation' (cutting off the decimal part 
    # toward zero), which differs from standard floor/ceil for negative numbers.
    divided_number = dividend / divisor
    divided_number = -int(-divided_number) if divided_number < 0 else int(divided_number)

    # Calculate the remainder: Remainder = Dividend - Divisor * Quotient
    remainder = dividend - divisor * divided_number

    return remainder

In [11]:
import math
import numpy as np

def transform_to_pipi(input_angle):
    """
    Unwinds an angle (in radians) to ensure it changes continuously
    and does not jump from +pi to -pi (or vice versa), which is vital for
    calculating a smooth derivative (rate of change of the angle).
    """
    pi = math.pi
    
    # Calculate how many full 2*pi revolutions have occurred
    revolutions = np.floor((input_angle + np.sign(input_angle) * pi) / (2 * pi))

    # Calculate the remainder of the angle after removing full revolutions
    p1 = truncated_remainder(input_angle + np.sign(input_angle) * pi, 2 * pi)
    
    # Apply a correction term (p2) based on the sign to correctly map the angle 
    # back into the range of [-pi, pi] (this handles the unwrapping logic)
    p2 = (np.sign(np.sign(input_angle)
                  + 2 * (np.sign(math.fabs((truncated_remainder(input_angle + pi, 2 * pi))
                                          / (2 * pi))) - 1))) * pi

    # The final unwrapped angle
    output_angle = p1 - p2

    return output_angle, revolutions

In [12]:
import math
import numpy as np

def remove_acceleration_outliers(acc):
    """
    Filters out extreme acceleration spikes by replacing values above a G-force
    threshold (default 7.0G) with the previous time step's value.
    """
    
    acc_threshold_g = 7.0  # F1 cars rarely sustain forces above 5-6G
    acc = np.array(acc)
    
    # Handle the very first point
    if (math.fabs(acc[0]) > acc_threshold_g):
        acc[0] = 0.0
        
    # Iterate through the main body of the data
    for i in range(1, acc.size-1):
        if ( math.fabs(acc[i]) > acc_threshold_g ):
            # Replace the outlier with the previous, clean value
            acc[i] = acc[i-1]
            
    # Handle the very last point
    if (acc.size > 1 and math.fabs(acc[-1]) > acc_threshold_g ):
        acc[-1] = acc[-2]
            
    return acc

In [13]:
import numpy as np
import math

def compute_accelerations(physics_df):
    """
    Calculates the Longitudinal (ax) and Lateral (ay) acceleration 
    (in G's) for every point in a telemetry lap.
    """
    
    # --- 1. Longitudinal Acceleration (ax) ---
    # Convert Speed from km/h to m/s for physics calculations
    v = np.array(physics_df['Speed']) / 3.6
    
    # Calculate the rate of change of speed over time (dv/dt) using the smooth derivative
    # Then divide by 9.81 m/s^2 to convert the units from m/s^2 to G-force.
    lon_acc = smooth_derivative(physics_df['Time'], v) / 9.81 

    # --- 2. Lateral Acceleration (ay) - Requires Heading & Curvature ---
    # Calculate the derivative of X and Y positions with respect to DISTANCE (dx/ds, dy/ds)
    # These derivatives define the tangent direction (the car's path)
    dx = smooth_derivative(physics_df['Distance'], physics_df['X'])
    dy = smooth_derivative(physics_df['Distance'], physics_df['Y'])

    # Initialize the array to store the car's heading angle (theta)
    theta = np.zeros(dx.size)
    
    # Determine the initial heading angle using atan2
    if dx.size > 0 and dy.size > 0:
        theta[0] = math.atan2(dy[0], dx[0])
    else:
        # Return zeros if telemetry is empty or too short
        physics_df['Longitudinal_G'] = np.zeros_like(v)
        physics_df['Lateral_G'] = np.zeros_like(v)
        return physics_df

    # Loop to calculate the continuous heading angle (theta)
    for i in range(1, dx.size):
        # Calculate the raw change in angle from the previous point
        delta_angle = math.atan2(dy[i], dx[i]) - theta[i-1]
        
        # Use transform_to_pipi to "unwind" this change, preventing jumps from +180 to -180
        unwound_delta, _ = transform_to_pipi(delta_angle)
        
        # Update the continuous heading angle
        theta[i] = theta[i-1] + unwound_delta
        
    # Curvature (kappa) is the rate of change of the continuous heading (theta) 
    # with respect to the Distance traveled (d(theta)/ds).
    kappa = smooth_derivative(physics_df['Distance'], theta)
    
    # Lateral Acceleration (ay) is calculated using the centripetal formula:
    # ay = v^2 * kappa. This is then converted to G-force.
    lat_acc = (v * v * kappa) / 9.81
    
    # --- 3. Outlier Removal ---
    # Apply the final filtering step to remove any extreme, non-physical spikes
    physics_df['Longitudinal_G'] = remove_acceleration_outliers(lon_acc)
    physics_df['Lateral_G'] = remove_acceleration_outliers(lat_acc)
    
    return physics_df

In [14]:
def compute_speeds(physics_df, fastest_lap):
    # Find max straight speed
    max_speed = physics_df['v_m/s'].max()
    physics_df['Max_Straight_Speed'] = max_speed

    # Find min corner speed
    # Define a "high-G corner" as the top 5% of all lateral G-forces
    HIGH_G_PERCENTILE = 95

    g_threshold = np.percentile(abs(physics_df['Lateral_G']), HIGH_G_PERCENTILE)
    high_g_corners = physics_df[abs(physics_df['Lateral_G']) >= g_threshold]
    min_corner_speed = high_g_corners['v_m/s'].min()
    physics_df['Min_Corner_Speed'] = min_corner_speed

    # Average lap speed
    total_distance_meters = physics_df['Distance'].max()
    total_time_seconds = fastest_lap['LapTime'].total_seconds()
    avg_speed_lap = total_distance_meters / total_time_seconds
    physics_df['Avg_Speed_Lap'] = avg_speed_lap

    return physics_df


In [15]:
def compute_forces(physics_df):
    # --- G-Force Features (Peak Performance) ---
    # maximum lateral G, car's maximum cornering grip
    #physics_df['LatG_max'] = max(abs(physics_df['Lateral_G']))
    physics_df['LatG_Max'] = physics_df['Lateral_G'].abs().max()

    # minimum longitudinal G, car's maximum braking force
    physics_df['LonG_Min'] = physics_df['Longitudinal_G'].min()
    # minimum longitudinal G, car's maximum acceleration
    physics_df['LonG_Max'] = physics_df['Longitudinal_G'].max()

    # --- Aero/Load Features (Setup & Car Characteristics) ---
    physics_df['DownForce_Max'] = physics_df['downForce_N'].max()
    physics_df['DownForce_Avg'] = physics_df['downForce_N'].mean()
    physics_df['DragForce_Max'] = physics_df['dragForce_N'].max()
    physics_df['LoadFront_Max'] = physics_df['loadFront_AeroModel_N'].max()
    physics_df['LoadRear_Max'] = physics_df['loadRear_AeroModel_N'].max()

    return physics_df

In [ ]:
def compute_base_data_df(fastest_lap, session, year, location):  
    # --- 1. Base Data & Weather Merge (From fastest_lap) ---
    lap_features = pd.DataFrame([fastest_lap.to_dict()])

    # Time conversions and setup
    lap_features['LapTime'] = lap_features['LapTime'].dt.total_seconds()
    lap_features['Sector1Time'] = lap_features['Sector1Time'].dt.total_seconds()
    lap_features['Sector2Time'] = lap_features['Sector2Time'].dt.total_seconds()
    lap_features['Sector3Time'] = lap_features['Sector3Time'].dt.total_seconds()
    lap_features['Location'] = location
    lap_features['Year'] = year

    lap_features['LapStartTime'] = lap_features['Time']
    weather = session.weather_data
    lap_features = pd.merge_asof(
        lap_features,                # Use the new DataFrame
        weather,
        left_on='LapStartTime', # Column from fl_df
        right_on='Time',        # Column from weather
        direction='backward'  # Finds weather row *just before* or at LapStartTime
    )
    lap_features = lap_features.rename(columns={'Time_x': 'Time'})
    lap_features = lap_features.drop(columns={'Time_y'})

    lap_features = pd.merge(lap_features,
                            circuit_info,
                            on='Location',
                            how='left')
    
    lap_features['EventDate'] = session.event['EventDate']

    # --- 2. Target Variable (Y) - OFFICIAL RANK ---
    results_df = session.results
    drv = lap_features['Driver'].iloc[0]
    rank_series = results_df[results_df['Abbreviation'] == drv]['Position']
    lap_features['Quali_Rank'] = rank_series.iloc[0] if not rank_series.empty else np.nan
    
    KEEP_COLS = ['Time', 'Driver', 'DriverNumber', 'Team', 'LapTime', 'LapNumber', 'Stint',
                  'Sector1Time', 'Sector2Time', 'Sector3Time', 'Compound', 'TyreLife', 'FreshTyre',
                  'AirTemp', 'TrackTemp', 'Rainfall', 'Humidity', 'WindSpeed', 'WindDirection', 
                  'TrackLenght', 'NumTurns', 'Traction', 'AsphaltGrip', 'AsphaltAbrasion', 
                  'TrackEvolution', 'TyreStress', 'Braking', 'LateralLoad', 'Downforce', 
                  'Soft', 'Medium', 'Hard', 'Location', 'Country', 'Year', 'EventDate', 'Quali_Rank'
                  ]
    
    lap_features = lap_features[KEEP_COLS]
    
    # Define ALL columns that will eventually be in the final DataFrame.
    ALL_COLS = [
        'Time', 'Driver', 'DriverNumber', 'Team', 'LapTime', 'LapNumber', 'Stint',
        'Sector1Time', 'Sector2Time', 'Sector3Time', 'Compound', 'TyreLife', 'FreshTyre',
        'AirTemp', 'TrackTemp', 'Rainfall', 'Humidity', 'WindSpeed', 'WindDirection', 
        'TrackLenght', 'NumTurns', 'Traction', 'AsphaltGrip', 'AsphaltAbrasion', 
        'TrackEvolution', 'TyreStress', 'Braking', 'LateralLoad', 'Downforce', 
        'Soft', 'Medium', 'Hard', 'Location', 'Country', 'Year', 'EventDate', 'Quali_Rank',
        # Physics Features (ALL physics columns are initialized to NaN)
        'Max_Straight_Speed', 'Avg_Lap_Speed', 'Min_Corner_Speed', 'Peak_Cornering_G', 
        'Peak_Braking_G', 'Peak_Acceleration_G', 'Peak_Downforce_N', 'Avg_Downforce_N', 
        'Peak_Drag_N', 'Peak_Front_Load_N', 'Peak_Rear_Load_N',
    ]
    
    # Get the list of columns currently in the DataFrame
    current_cols = lap_features.columns.tolist()
    
    # Add all missing physics and info columns and set them to NaN
    for col in ALL_COLS:
        if col not in current_cols:
            lap_features[col] = np.nan
            
    # Filter to ensure order and only necessary columns remain
    final_base_cols = [col for col in ALL_COLS if col in lap_features.columns]
    
    return lap_features[final_base_cols].copy()


In [83]:
def add_physics_features(lap_features_df, physics_df, fastest_lap):
    """
    Calculates the physics aggregations and updates the existing DataFrame.
    """
    # --- Speed Features ---
    lap_features_df['Max_Straight_Speed'] = physics_df['v_m/s'].max()
    total_distance_meters = physics_df['Distance'].max()
    total_time_seconds = fastest_lap['LapTime'].total_seconds()
    lap_features_df['Avg_Lap_Speed'] = total_distance_meters / total_time_seconds

    HIGH_G_PERCENTILE = 95
    g_threshold = np.percentile(abs(physics_df['Lateral_G']), HIGH_G_PERCENTILE)
    high_g_corners = physics_df[abs(physics_df['Lateral_G']) >= g_threshold]
    lap_features_df['Min_Corner_Speed'] = high_g_corners['v_m/s'].min()

    # --- G-Force Features (Peak Performance) ---
    lap_features_df['Peak_Cornering_G'] = abs(physics_df['Lateral_G']).max()
    lap_features_df['Peak_Braking_G'] = physics_df['Longitudinal_G'].min()
    lap_features_df['Peak_Acceleration_G'] = physics_df['Longitudinal_G'].max()

    # --- Aero/Load Features (Your selected columns) ---
    lap_features_df['Peak_Downforce_N'] = physics_df['downForce_N'].max()
    lap_features_df['Avg_Downforce_N'] = physics_df['downForce_N'].mean()
    lap_features_df['Peak_Drag_N'] = physics_df['dragForce_N'].max()
    lap_features_df['Peak_Front_Load_N'] = physics_df['loadFront_AeroModel_N'].max()
    lap_features_df['Peak_Rear_Load_N'] = physics_df['loadRear_AeroModel_N'].max()

    return lap_features_df

# Building dataframe

In [110]:
YEAR = 2022
LOCATION = 'Zandvoort'

session = ff1.get_session(2022, 'Zandvoort', 'Q')
session.load()
driver = 'VER'
# fastest lap of the driver
fastest_lap = session.laps.pick_drivers('23').pick_fastest()

lap_features_df = compute_base_data_df(fastest_lap, session, YEAR, LOCATION)
try:
    telemetry_driver = fastest_lap.get_telemetry().add_distance()


    # Create a new DataFrame to calcualate data
    physics_df = telemetry_driver.copy()
    physics_df = calculate_physics(physics_df)
    physics_df = compute_accelerations(physics_df)

    lap_features_df = add_physics_features(lap_features_df, physics_df, fastest_lap)
except Exception as e:
    print(f"Skipped {LOCATION} {YEAR}: {e}")



Skipped Zandvoort 2022: attempt to get argmin of an empty sequence


In [111]:
lap_features_df

,Time,Driver,DriverNumber,Team,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,...,Avg_Lap_Speed,Min_Corner_Speed,Peak_Cornering_G,Peak_Braking_G,Peak_Acceleration_G,Peak_Downforce_N,Avg_Downforce_N,Peak_Drag_N,Peak_Front_Load_N,Peak_Rear_Load_N
0,0 days 00:30:24.229000,ALB,23,Williams,71.695,8.0,2.0,24.606,25.238,21.851,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
results_df = session.results
rank_series = results_df[results_df['Abbreviation'] == driver]['Position']

In [88]:
rank_series

1    1.0
Name: Position, dtype: float64

In [20]:
lap_features

,Time,Driver,DriverNumber,Team,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,...,Avg_Lap_Speed,Min_Corner_Speed,Peak_Cornering_G,Peak_Braking_G,Peak_Acceleration_G,Peak_Downforce_N,Avg_Downforce_N,Peak_Drag_N,Peak_Front_Load_N,Peak_Rear_Load_N
0,0 days 01:14:52.170000,VER,1,Red Bull Racing,78.792,17.0,6.0,26.262,26.483,26.047,...,73.001475,52.777778,6.662891,-5.410596,1.928182,36000.640278,22274.462437,9729.902778,18120.694123,27201.10299


In [42]:
session = ff1.get_session(2022, 'Monza', 'Q')
session.load()

core           INFO 	Loading data for Italian Grand Prix - Qualifying [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '55', '11', '44', '63', '4', '3', '10', '14', '31', '77', '45', '24', '22', '6', '5', '18', '20', '47']


In [ ]:
run_config = [
    (2022, 2, None),  # Per [2:]
    (2023, 1, None),  # Per [1:]
    (2024, 1, None),  # Per [1:]
    (2025, 1, 20)    # Per [1:20]
]

for YEAR, START, END in run_config:
    locations = ff1.get_event_schedule(YEAR)['Location'][START:END]

    all_qualis = []

    for LOCATION in locations:
        try:
            if LOCATION == 'Miami Gardens':
                LOCATION = 'Miami'

            session = ff1.get_session(YEAR, LOCATION, 'Q')
            session.load()
            drivers = session.drivers
            event_laps = []

            for driver_number in drivers:
                fastest_lap = session.laps.pick_drivers(driver_number).pick_fastest()
                lap_features_df = compute_base_data_df(fastest_lap, session, YEAR, LOCATION)

                try:
                    telemetry_driver = fastest_lap.get_telemetry().add_distance()

                    physics_df = telemetry_driver.copy()
                    physics_df = calculate_physics(physics_df)
                    physics_df = compute_accelerations(physics_df)
                    lap_features_df = add_physics_features(lap_features_df, physics_df, fastest_lap)

                except Exception as e:
                    print(f"Skipped physiscs {LOCATION} {YEAR} {driver_number}: {e}")

                event_laps.append(lap_features_df)

            el_df = pd.concat(event_laps, ignore_index=True)
            all_qualis.append(el_df)

        except Exception as e:
            print(f"Skipped {LOCATION} {YEAR}: {e}")


all_qualis_df = pd.concat(all_qualis, ignore_index=True)

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '55', '11', '44', '77', '20', '14', '63', '10', '31', '47', '4', '23', '24', '22', '27', '3', '18', '6']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Qualifying [v3.6.1]
req            INFO 	Usi

Skipped Jeddah 2022: 'NoneType' object has no attribute 'get_telemetry'


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '11', '4', '44', '63', '3', '31', '55', '14', '10', '77', '22', '24', '47', '23', '20', '5', '6', '18']
core           INFO 	Loading data for Emilia Romagna Grand Prix - Qualifying [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Skipped Melbourne 2022: 'NoneType' object has no attribute 'get_telemetry'


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '4', '20', '14', '3', '11', '77', '5', '55', '63', '47', '44', '24', '18', '22', '10', '6', '31', '23']
core           INFO 	Loading data for Miami Grand Prix - Qualifying [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


...

Skipped Imola 2025: 'NoneType' object has no attribute 'get_telemetry'


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '16', '81', '44', '1', '6', '14', '31', '30', '23', '55', '22', '27', '63', '12', '5', '87', '10', '18', '43']
core           INFO 	Loading data for Spanish Grand Prix - Qualifying [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data


Skipped Zandvoort 2025: 'NoneType' object has no attribute 'get_telemetry'


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '81', '16', '44', '63', '12', '5', '14', '22', '87', '27', '55', '23', '31', '6', '18', '43', '10', '30']
core           INFO 	Loading data for Azerbaijan Grand Prix - Qualifying [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_da

Skipped Austin 2025: 'NoneType' object has no attribute 'get_telemetry'


In [47]:
all_qualis_df

,Time,Driver,DriverNumber,Team,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,...,Min_Corner_Speed,Peak_Cornering_G,Peak_Braking_G,Peak_Acceleration_G,Peak_Downforce_N,Avg_Downforce_N,Peak_Drag_N,Peak_Front_Load_N,Peak_Rear_Load_N,Rank
0,0 days 01:13:05.010000,PIA,81,McLaren,90.641,19.0,6.0,23.996,27.227,39.418,...,46.944444,6.830831,-4.629116,1.960726,32372.751736,15282.021557,8749.392361,16411.966446,24914.832800,1
1,0 days 01:13:46.450000,RUS,63,Mercedes,90.723,21.0,6.0,24.068,27.221,39.434,...,51.111111,6.986890,-5.568742,2.027918,33361.175974,15423.313207,9016.534047,16918.514866,25526.422595,2
2,0 days 00:43:58.208000,NOR,4,McLaren,90.787,11.0,3.0,23.954,27.257,39.576,...,35.286458,6.701248,-4.154773,1.902269,32176.850511,15339.026035,8696.446084,15951.252036,24742.605183,3
3,0 days 01:12:31.500000,VER,1,Red Bull Racing,90.817,14.0,5.0,23.945,27.313,39.559,...,50.966435,6.919067,-4.514572,1.648351,33560.644444,15579.638276,9070.444444,17116.480298,25650.515978,4
4,0 days 01:05:18.565000,HAM,44,Ferrari,90.927,17.0,5.0,24.148,27.175,39.604,...,32.777778,6.514573,-6.391433,1.679845,32766.337809,15310.559417,8855.766975,16881.265327,25137.122713,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,0 days 00:30:49.791000,BOR,5,Kick Sauber,90.820,7.0,2.0,26.952,38.250,25.618,...,24.444444,6.754018,-4.576611,1.909166,29496.660156,12652.616719,7972.070312,15394.823719,23002.128718,16
276,0 days 00:28:37.456000,STR,18,Aston Martin,90.949,8.0,2.0,26.911,38.102,25.936,...,22.777778,6.396420,-5.487956,2.783474,29872.410002,12437.507369,8073.624325,15332.338330,23344.780450,17
277,0 days 00:29:41,COL,43,Alpine,90.982,8.0,3.0,26.776,38.309,25.897,...,24.166667,6.446463,-4.596947,2.152730,29872.410002,12396.744189,8073.624325,15419.758422,23427.608176,18
278,0 days 00:31:00.841000,OCO,31,Haas F1 Team,90.989,5.0,2.0,26.931,38.465,25.593,...,18.829862,6.217829,-5.295989,2.258368,29684.237809,12613.585862,8022.766975,15734.634972,23167.524003,19


In [48]:
all_qualis_df.to_csv('quali_telemetry.csv', index=False)

In [49]:
import logging
logging.getLogger('fastf1').setLevel(logging.WARNING)

In [52]:
all_qualis_df

,Time,Driver,DriverNumber,Team,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,...,Min_Corner_Speed,Peak_Cornering_G,Peak_Braking_G,Peak_Acceleration_G,Peak_Downforce_N,Avg_Downforce_N,Peak_Drag_N,Peak_Front_Load_N,Peak_Rear_Load_N,Rank
0,0 days 01:14:47.968000,LEC,16,Ferrari,90.558,14.0,5.0,29.115,38.702,22.741,...,35.033563,6.315997,-5.639210,1.655335,30061.176736,15110.574496,8124.642361,15967.697325,23405.277478,1
1,0 days 01:15:15.614000,VER,1,Red Bull Racing,90.681,13.0,4.0,28.970,38.832,22.879,...,24.943577,6.049043,-4.409588,2.149994,30822.189082,14709.982510,8330.321373,15830.695983,23927.015267,2
2,0 days 01:07:34.650000,SAI,55,Ferrari,90.687,11.0,4.0,29.036,38.842,22.809,...,34.722222,6.617073,-5.031731,2.238076,29872.410002,14717.819360,8073.624325,15139.076301,23294.986842,3
3,0 days 01:15:21.593000,PER,11,Red Bull Racing,90.921,17.0,6.0,29.180,38.894,22.847,...,36.388889,6.820775,-4.622779,1.671452,31013.928520,14899.578713,8382.142843,16030.054736,24011.176938,4
4,0 days 00:55:33.904000,HAM,44,Mercedes,91.048,10.0,3.0,29.100,39.038,22.910,...,26.944444,6.203374,-4.674010,1.668267,29872.410002,14982.631982,8073.624325,14955.892800,23263.427286,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,0 days 00:29:21.710000,MAG,20,Haas F1 Team,85.834,8.0,2.0,17.637,36.876,31.321,...,28.297619,6.285979,-4.379501,1.769746,31786.831684,15536.006809,8591.035590,15927.352317,24504.781940,16
256,0 days 00:21:22.638000,GAS,10,AlphaTauri,85.859,5.0,2.0,17.559,36.854,31.446,...,45.833333,6.894909,-4.486195,1.458548,31399.191020,15715.337850,8486.267843,15664.977358,24288.403388,17
257,0 days 00:28:54.123000,BOT,77,Alfa Romeo,85.892,5.0,2.0,17.808,36.913,31.171,...,27.111111,6.884178,-4.922296,1.582875,31206.262500,15511.837413,8434.125000,16011.351288,24135.136841,18
258,0 days 00:28:48.926000,ALB,23,Williams,86.028,8.0,2.0,17.561,36.708,31.759,...,26.833333,6.745130,-4.358507,1.669836,32964.022656,15745.752053,8909.195312,16786.648806,25273.686130,19


In [53]:
all_qualis_df['Location'].unique()

array(['Sakhir', 'Barcelona', 'Monaco', 'Montréal', 'Silverstone',
       'Spielberg', 'Budapest', 'Spa-Francorchamps', 'Monza',
       'Marina Bay', 'Austin', 'São Paulo', 'Yas Island'], dtype=object)

In [51]:
all_qualis_df.to_csv('quali_telemetry.csv', index=False)

In [55]:
session = ff1.get_session(2022, 'Jeddah', 'Q')
session.load()
session.drivers

logger      WARNING 	Failed to load race control messages!


['11',
 '16',
 '55',
 '1',
 '31',
 '63',
 '14',
 '77',
 '10',
 '20',
 '4',
 '3',
 '24',
 '47',
 '18',
 '44',
 '23',
 '27',
 '6',
 '22']

In [ ]:
YEAR = 2022
locations = ff1.get_event_schedule(YEAR)['Location'][START:END]

for LOCATION in locations:
    
    try:
        session = ff1.get_session(YEAR, LOCATION, 'Q')
        session.load()
        drivers = session.drivers
        event_laps = []

        for rank, driver_number in enumerate(drivers):
            print('Driver: ' + driver_number)
            fastest_lap = session.laps.pick_drivers(driver_number).pick_fastest()
            telemetry_driver = fastest_lap.get_telemetry().add_distance()

    except Exception as e:
        print(f"Skipped {LOCATION} {YEAR}: {e}")

In [72]:
YEAR = 2022
LOCATION = 'Zandvort'

session = ff1.get_session(YEAR, LOCATION, 'Q')
session.load()
drivers = session.drivers
print(drivers)
print('------')
try:
    for rank, driver_number in enumerate(drivers):
        print('Driver: ' + driver_number)
        fastest_lap = session.laps.pick_drivers(driver_number).pick_fastest()
        try:
            telemetry_driver = fastest_lap.get_telemetry().add_distance()

            physics_df = telemetry_driver.copy()
            physics_df = calculate_physics(physics_df)
            physics_df = compute_accelerations(physics_df)
            lap_features = compute_data_df(physics_df, fastest_lap, session, YEAR, LOCATION)
            lap_features['Rank'] = rank+1
        except Exception as e:
            print(f"Driver problem: {driver_number}, {e}")
except Exception as e:
    print(f"Skipped {LOCATION} {YEAR}: {e}")

events      WARNING 	Correcting user input 'Zandvort' to 'Dutch Grand Prix'


['1', '16', '55', '44', '11', '63', '4', '47', '22', '18', '10', '31', '14', '24', '23', '77', '3', '20', '5', '6']
------
Driver: 1
Driver: 16
Driver: 55
Driver: 44
Driver: 11
Driver: 63
Driver: 4
Driver: 47
Driver: 22
Driver: 18
Driver: 10
Driver: 31
Driver: 14
Driver: 24
Driver: 23
Driver problem: 23, attempt to get argmin of an empty sequence
Driver: 77
Driver: 3
Driver: 20
Driver: 5
Driver: 6


In [81]:
session.results


,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,...,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Laps
1,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,1e5bc6,red_bull,Max,Verstappen,Max Verstappen,...,1.0,,NaN,0 days 00:01:11.317000,0 days 00:01:10.927000,0 days 00:01:10.342000,NaT,,NaN,NaN
16,16,C LECLERC,LEC,leclerc,Ferrari,ed1c24,ferrari,Charles,Leclerc,Charles Leclerc,...,2.0,,NaN,0 days 00:01:11.443000,0 days 00:01:10.988000,0 days 00:01:10.363000,NaT,,NaN,NaN
55,55,C SAINZ,SAI,sainz,Ferrari,ed1c24,ferrari,Carlos,Sainz,Carlos Sainz,...,3.0,,NaN,0 days 00:01:11.767000,0 days 00:01:10.814000,0 days 00:01:10.434000,NaT,,NaN,NaN
44,44,L HAMILTON,HAM,hamilton,Mercedes,6cd3bf,mercedes,Lewis,Hamilton,Lewis Hamilton,...,4.0,,NaN,0 days 00:01:11.331000,0 days 00:01:11.075000,0 days 00:01:10.648000,NaT,,NaN,NaN
11,11,S PEREZ,PER,perez,Red Bull Racing,1e5bc6,red_bull,Sergio,Perez,Sergio Perez,...,5.0,,NaN,0 days 00:01:11.641000,0 days 00:01:11.314000,0 days 00:01:11.077000,NaT,,NaN,NaN
63,63,G RUSSELL,RUS,russell,Mercedes,6cd3bf,mercedes,George,Russell,George Russell,...,6.0,,NaN,0 days 00:01:11.561000,0 days 00:01:10.824000,0 days 00:01:11.147000,NaT,,NaN,NaN
4,4,L NORRIS,NOR,norris,McLaren,f58020,mclaren,Lando,Norris,Lando Norris,...,7.0,,NaN,0 days 00:01:11.556000,0 days 00:01:11.116000,0 days 00:01:11.174000,NaT,,NaN,NaN
47,47,M SCHUMACHER,MSC,mick_schumacher,Haas F1 Team,b6babd,haas,Mick,Schumacher,Mick Schumacher,...,8.0,,NaN,0 days 00:01:11.741000,0 days 00:01:11.420000,0 days 00:01:11.442000,NaT,,NaN,NaN
22,22,Y TSUNODA,TSU,tsunoda,AlphaTauri,4e7c9b,alphatauri,Yuki,Tsunoda,Yuki Tsunoda,...,9.0,,NaN,0 days 00:01:11.427000,0 days 00:01:11.428000,0 days 00:01:12.556000,NaT,,NaN,NaN
18,18,L STROLL,STR,stroll,Aston Martin,2d826d,aston_martin,Lance,Stroll,Lance Stroll,...,10.0,,NaN,0 days 00:01:11.568000,0 days 00:01:11.416000,NaT,NaT,,NaN,NaN


In [79]:
# Use try/except or a robust filter as not all drivers in 'drivers' list may have a rank
rank_series = results_df[results_df['Abbreviation'] == driver_number]['Position']
if not rank_series.empty:
    lap_features['Quali_Rank'] = rank_series.iloc[0]
else:
    print(f"-> WARNING: Could not find official rank for {driver_number}. Skipping.") 

-> WARNING: Could not find official rank for 6. Skipping.
